In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

In [54]:
df_Ecommerce = pd.read_csv("D:/02_DATA/02_MINDX/01_MODULE_2/BUOI_4/Ecommerce.csv", encoding="unicode_escape")

In [55]:
# Get the summary info of data set
def table_summary(df):
    print(f"Dataset shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes, columns=["dtypes"])
    summary = summary.reset_index()
    summary["Name"] = summary["index"]
    summary = summary[['Name','dtypes']]
    summary["Missing value"] = df.isnull().sum().values
    summary["Missing value %"] = 100 * np.round(df.isnull().sum().values / len(df), 2)
    summary["Duplicate value"] = df.duplicated().sum()
    summary["Duplicate value %"] = 100 * np.round(df.duplicated().sum() / len(df), 2)
    summary['Uniques'] = df.nunique().values
    return summary

In [56]:
table_summary(df=df_Ecommerce)

Dataset shape: (541909, 8)


,Name,dtypes,Missing value,Missing value %,Duplicate value,Duplicate value %,Uniques
0,InvoiceNo,object,0,0.0,5268,1.0,25900
1,StockCode,object,0,0.0,5268,1.0,4070
2,Description,object,1454,0.0,5268,1.0,4223
3,Quantity,int64,0,0.0,5268,1.0,722
4,InvoiceDate,object,0,0.0,5268,1.0,23260
5,UnitPrice,float64,0,0.0,5268,1.0,1630
6,CustomerID,float64,135080,25.0,5268,1.0,4372
7,Country,object,0,0.0,5268,1.0,38


The column `CustomerID` is just number to identify this student in this dataset. So this column is not really a part of the information we should care about. We can drop this column or make it the index for this dataset.

In [57]:
# df_Ecommerce.drop("CustomerID", axis=1, inplace=True)

In [58]:
table_summary(df=df_Ecommerce)

Dataset shape: (541909, 8)


,Name,dtypes,Missing value,Missing value %,Duplicate value,Duplicate value %,Uniques
0,InvoiceNo,object,0,0.0,5268,1.0,25900
1,StockCode,object,0,0.0,5268,1.0,4070
2,Description,object,1454,0.0,5268,1.0,4223
3,Quantity,int64,0,0.0,5268,1.0,722
4,InvoiceDate,object,0,0.0,5268,1.0,23260
5,UnitPrice,float64,0,0.0,5268,1.0,1630
6,CustomerID,float64,135080,25.0,5268,1.0,4372
7,Country,object,0,0.0,5268,1.0,38


In [59]:
# Replace all missing values (NaN) in the director column of df_student with the most common value (mode) from that column.
feature_missing = ["Description", "CustomerID"]
for feature in feature_missing:
    df_Ecommerce[feature] = df_Ecommerce[feature].fillna(df_Ecommerce[feature].mode()[0])

In [60]:
table_summary(df=df_Ecommerce)

Dataset shape: (541909, 8)


,Name,dtypes,Missing value,Missing value %,Duplicate value,Duplicate value %,Uniques
0,InvoiceNo,object,0,0.0,5268,1.0,25900
1,StockCode,object,0,0.0,5268,1.0,4070
2,Description,object,0,0.0,5268,1.0,4223
3,Quantity,int64,0,0.0,5268,1.0,722
4,InvoiceDate,object,0,0.0,5268,1.0,23260
5,UnitPrice,float64,0,0.0,5268,1.0,1630
6,CustomerID,float64,0,0.0,5268,1.0,4372
7,Country,object,0,0.0,5268,1.0,38


In [61]:
def checking_outlier(list_feature, df = df_Ecommerce):
    outlier_info = []
    for feature in list_feature:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)][feature]
        if len(outliers) == 0:
            outlier_detail = ""
        else:
            outlier_detail = outliers.tolist()
        outlier_info.append({
            "Feature": feature,
            "Outlier Count": len(outliers),
            "Outlier Detail": outlier_detail
        })
    return pd.DataFrame(outlier_info)

In [62]:
df_Ecommerce.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,541909.000000
mean,9.552250,4.611114,15924.146207
std,218.081158,96.759853,1850.531104
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,14367.000000
50%,3.000000,2.080000,16249.000000
75%,10.000000,4.130000,17841.000000
max,80995.000000,38970.000000,18287.000000


In [63]:
checking_outlier(list_feature=["Quantity", "UnitPrice"])

,Feature,Outlier Count,Outlier Detail
0,Quantity,58619,"[32, 24, 24, 24, 48, 24, 24, 24, 24, 24, 24, 2..."
1,UnitPrice,39627,"[9.95, 18.0, 10.95, 27.5, 14.95, 14.95, 16.95,..."


In [65]:
df_Ecommerce['TotalPrice'] = df_Ecommerce['UnitPrice'] * df_Ecommerce['Quantity']

In [66]:
df_Ecommerce['InvoiceDate'] = pd.to_datetime (df_Ecommerce['InvoiceDate'], format = 'mixed')

In [67]:
df_Ecommerce['Status'] = 'Completed'
df_Ecommerce.loc[df_Ecommerce['Quantity'] <= 0, 'Status'] = 'Cancelled'
df_Ecommerce

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Status
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,Completed
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Completed
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,Completed
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Completed
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Completed
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,Completed
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,Completed
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,Completed
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,Completed


In [68]:
df_Ecommerce.groupby (by = 'Status', as_index = False)\
    .agg (InvoiceCount = ('InvoiceNo', 'nunique'))

,Status,InvoiceCount
0,Cancelled,5172
1,Completed,20728


In [76]:
# Đếm số lượng đơn hàng theo từng quốc gia (country)
df_Ecommerce.groupby (by = 'Country', as_index = False)\
    .agg (InvoiceCount = ('InvoiceNo', 'nunique'))\
    .sort_values (by = 'InvoiceCount', ascending = False)

,Country,InvoiceCount
36,United Kingdom,23494
14,Germany,603
13,France,461
10,EIRE,360
3,Belgium,119
31,Spain,105
24,Netherlands,101
33,Switzerland,74
27,Portugal,71
0,Australia,69


In [ ]:
# Tính Doanh số của từng quốc gia, chỉ lấy những đơn hàng đã hoàn tất
completed_orders = df_Ecommerce[df_Ecommerce["Status"] == "Completed"]

revenue_by_country = completed_orders.groupby("Country")["TotalPrice"].sum()
revenue_by_country

Country
Australia                138521.310
Austria                   10198.680
Bahrain                     754.140
Belgium                   41196.340
Brazil                     1143.600
Canada                     3666.380
Channel Islands           20450.440
Cyprus                    13590.380
Czech Republic              826.740
Denmark                   18955.340
EIRE                     283453.960
European Community         1300.250
Finland                   22546.080
France                   209715.110
Germany                  228867.140
Greece                     4760.520
Hong Kong                 15691.800
Iceland                    4310.000
Israel                     8135.260
Italy                     17483.240
Japan                     37416.370
Lebanon                    1693.880
Lithuania                  1661.060
Malta                      2725.590
Netherlands              285446.340
Norway                    36165.440
Poland                     7334.650
Portugal            

In [ ]:
# tính số lượng sản phẩm bán ra của năm 2011 ở nước Anh
sales_2011_uk = df_Ecommerce[(df_Ecommerce["Country"] == "United Kingdom") & (df_Ecommerce['InvoiceDate'].dt.year == 2011)]

total_quantity_2011_uk = sales_2011_uk["Quantity"].sum()
total_quantity_2011_uk

np.int64(3965728)